# Utilisation de l'api SNCF

Ce notebook a pour but d'utiliser l'api SNCF afin de répondre a la question suivante : 

Dans quelle gare il y a le plus de retard de train en France.

Pour utiliser l'api nous avons besoin d'une clef API qui sera stocker dans un secret.


__Librairies :__

python-dotenv

In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json

Je charge ma clé API dans une variable

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [3]:
def requete_api(requete):
    reponse = requests.get(requete, auth=(api_key,""))
    reponse.status_code
    reponse_json = reponse.json()
    return reponse_json

In [4]:
def ecrire_json(data,nom_fichier):
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [5]:
def lire_avec_pandas(json):
    dataframe = pd.read_json(json)
    return dataframe

In [6]:
def main():
    pass

### Exemples

Le détail des modes de transport couvert par l’API

In [7]:
requete = "https://api.sncf.com/v1/coverage/sncf/commercial_modes"
reponse = requete_api(requete)
ecrire_json(reponse,"reponse.json")
# df = pd.read_json(reponse, orient="index")